In [ ]:
"""
OPTIMIZED: Train Double DQN RL Agent with Episode Length Limits
Key changes:
1. Added max_steps_per_episode parameter
2. Added progress tracking within episodes
3. Reduced default episodes but each completes faster
4. Added early stopping
"""

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
from pathlib import Path
from collections import deque
import random
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from datetime import datetime

# ============================================================
# PATHS
# ============================================================
RL_INPUT_PATH = Path("C:/Users/wdkal/Downloads/RL_INPUTS")
OUTPUT_PATH = Path("C:/Users/wdkal/Downloads/RL_OUTPUTS")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

# ============================================================
# OPTIMIZED TRADING ENVIRONMENT
# ============================================================

class TradingEnvironment:
    """Trading environment with episode length limits"""
    def __init__(self, rl_input_path, transaction_cost=0.001, max_steps_per_episode=5000):
        with open(rl_input_path, 'rb') as f:
            data = pickle.load(f)
        
        self.xgb_preds = data['predictions']['xgb']
        self.lstm_preds = data['predictions']['lstm']
        self.tcn_preds = data['predictions']['tcn']
        self.transformer_preds = data['predictions']['transformer']
        self.actual_labels = data['actual_labels']
        
        self.num_events = len(self.actual_labels)
        self.transaction_cost = transaction_cost
        self.max_steps_per_episode = max_steps_per_episode
        
        print(f"  Loaded {self.num_events:,} events")
        print(f"  Max steps per episode: {self.max_steps_per_episode:,}")
        
        self.reset()
    
    def reset(self, random_start=True):
        """Reset environment with optional random starting point"""
        if random_start and self.num_events > self.max_steps_per_episode:
            # Start at random position to see different market conditions
            max_start = self.num_events - self.max_steps_per_episode
            self.start_step = random.randint(0, max_start)
        else:
            self.start_step = 0
        
        self.current_step = self.start_step
        self.episode_step = 0  # Steps within this episode
        self.position = 0.0
        self.pnl = 0.0
        self.pnl_history = [0.0]
        self.trades = []
        self.position_history = [0.0]
        return self.get_state()
    
    def get_state(self):
        """Get current state (28 features)"""
        if self.current_step >= self.num_events:
            return None
        
        # Model predictions (12 features)
        xgb = self.xgb_preds[self.current_step]
        lstm = self.lstm_preds[self.current_step]
        tcn = self.tcn_preds[self.current_step]
        transformer = self.transformer_preds[self.current_step]
        
        # Ensemble stats (4 features)
        all_probs = np.array([xgb, lstm, tcn, transformer])
        mean_up = np.mean(all_probs[:, 2])
        mean_neutral = np.mean(all_probs[:, 1])
        mean_down = np.mean(all_probs[:, 0])
        std_up = np.std(all_probs[:, 2])
        
        # Market features (5 features)
        price_trend = mean_up - mean_down
        confidence = max(mean_up, mean_neutral, mean_down)
        uncertainty = std_up
        non_neutral = mean_up + mean_down
        time_progress = float(self.episode_step) / self.max_steps_per_episode
        
        # Trading state (7 features)
        num_recent_trades = len([t for t in self.trades if t['step'] > self.episode_step - 100])
        recent_pnl = self.pnl_history[-1]
        
        state = np.concatenate([
            xgb, lstm, tcn, transformer,  # 12
            [mean_up, mean_neutral, mean_down, std_up],  # 4
            [price_trend, confidence, uncertainty, non_neutral, time_progress],  # 5
            [self.position, recent_pnl/100, num_recent_trades/10, 
             self.episode_step/self.max_steps_per_episode, abs(self.position),
             1.0 if self.position > 0 else 0.0, 1.0 if self.position < 0 else 0.0]  # 7
        ])
        
        return state.astype(np.float32)
    
    def step(self, action):
        """Take action and return next state, reward, done"""
        # Action mapping
        action_map = {0: -0.5, 1: -0.3, 2: -0.1, 3: 0.0, 4: 0.1, 5: 0.3, 6: 0.5}
        position_change = action_map[action]
        
        old_position = self.position
        new_position = np.clip(self.position + position_change, -1.0, 1.0)
        actual_change = new_position - old_position
        
        # Transaction cost
        cost = abs(actual_change) * self.transaction_cost * 100
        
        self.position = new_position
        self.position_history.append(self.position)
        self.current_step += 1
        self.episode_step += 1
        
        # Check if episode should end
        done = (self.episode_step >= self.max_steps_per_episode or 
                self.current_step >= self.num_events)
        
        reward = 0
        
        if self.current_step < self.num_events:
            # Price movement
            actual_label = self.actual_labels[self.current_step]
            price_movement = (actual_label - 1) * 0.5
            
            # PnL calculation
            pnl = self.position * price_movement * 100
            reward = pnl - cost - 0.1 * (self.position ** 2)
            
            self.pnl += reward
            self.pnl_history.append(self.pnl)
            
            # Track trades
            if actual_change != 0:
                self.trades.append({
                    'step': self.episode_step,
                    'action': action,
                    'position': self.position,
                    'pnl': reward
                })
        
        next_state = self.get_state() if not done else None
        return next_state, reward, done, {}
    
    def get_metrics(self):
        """Calculate performance metrics"""
        if len(self.pnl_history) < 2:
            return {'total_pnl': 0, 'sharpe': 0, 'num_trades': 0, 'max_drawdown': 0}
        
        total_pnl = self.pnl_history[-1]
        returns = np.diff(self.pnl_history)
        mean_return = np.mean(returns)
        std_return = np.std(returns) if len(returns) > 1 else 1
        
        sharpe = (mean_return / std_return) * np.sqrt(len(returns)) if std_return > 0 else 0
        
        # Max drawdown
        cummax = np.maximum.accumulate(self.pnl_history)
        drawdown = np.array(self.pnl_history) - cummax
        max_drawdown = np.min(drawdown)
        
        return {
            'total_pnl': total_pnl,
            'num_trades': len(self.trades),
            'sharpe': sharpe,
            'max_drawdown': max_drawdown
        }

# ============================================================
# DOUBLE DQN AGENT (Same as before)
# ============================================================

class QNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(QNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(state_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, action_dim)
        )
    
    def forward(self, state):
        return self.network(state)

class DoubleDQN:
    def __init__(self, state_dim=28, action_dim=7, learning_rate=0.001,
                 gamma=0.95, epsilon_start=1.0, epsilon_end=0.01,
                 epsilon_decay=0.995, buffer_size=10000):
        
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.gamma = gamma
        self.epsilon = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        
        self.policy_net = QNetwork(state_dim, action_dim)
        self.target_net = QNetwork(state_dim, action_dim)
        self.target_net.load_state_dict(self.policy_net.state_dict())
        
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=learning_rate)
        self.criterion = nn.MSELoss()
        
        self.replay_buffer = deque(maxlen=buffer_size)
        self.losses = []
    
    def select_action(self, state, training=True):
        if training and random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        
        with torch.no_grad():
            state_tensor = torch.FloatTensor(state).unsqueeze(0)
            q_values = self.policy_net(state_tensor)
            return q_values.argmax().item()
    
    def remember(self, state, action, reward, next_state, done):
        self.replay_buffer.append((state, action, reward, next_state, done))
    
    def train_step(self, batch_size=64):
        if len(self.replay_buffer) < batch_size:
            return None
        
        batch = random.sample(self.replay_buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        
        states = torch.FloatTensor(np.array(states))
        actions = torch.LongTensor(actions)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(np.array([s if s is not None else np.zeros(self.state_dim) 
                                                    for s in next_states]))
        dones = torch.FloatTensor(dones)
        
        current_q = self.policy_net(states).gather(1, actions.unsqueeze(1))
        
        with torch.no_grad():
            next_actions = self.policy_net(next_states).argmax(1)
            next_q = self.target_net(next_states).gather(1, next_actions.unsqueeze(1)).squeeze()
            target_q = rewards + (1 - dones) * self.gamma * next_q
        
        loss = self.criterion(current_q.squeeze(), target_q)
        
        self.optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(self.policy_net.parameters(), 1.0)
        self.optimizer.step()
        
        self.losses.append(loss.item())
        return loss.item()
    
    def update_target_network(self):
        self.target_net.load_state_dict(self.policy_net.state_dict())
    
    def decay_epsilon(self):
        self.epsilon = max(self.epsilon_end, self.epsilon * self.epsilon_decay)
    
    def save(self, path):
        torch.save({
            'policy_net': self.policy_net.state_dict(),
            'target_net': self.target_net.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'epsilon': self.epsilon
        }, path)
    
    def load(self, path):
        checkpoint = torch.load(path, map_location='cpu')
        self.policy_net.load_state_dict(checkpoint['policy_net'])
        self.target_net.load_state_dict(checkpoint['target_net'])
        self.optimizer.load_state_dict(checkpoint['optimizer'])
        self.epsilon = checkpoint['epsilon']

# ============================================================
# TRAINING WITH PROGRESS TRACKING
# ============================================================

def validate(agent, env):
    """Run validation episode"""
    state = env.reset(random_start=False)  # Always start from beginning for validation
    while state is not None:
        action = agent.select_action(state, training=False)
        state, _, done, _ = env.step(action)
        if done:
            break
    return env.get_metrics()

print("="*60)
print("OPTIMIZED RL AGENT TRAINING")
print("="*60)

# Load environments with episode limits
print("\nLoading data...")
MAX_STEPS = 5000  # Limit each episode to 5000 steps for faster training
train_env = TradingEnvironment(RL_INPUT_PATH / "rl_input_train.pkl", 
                               max_steps_per_episode=MAX_STEPS)
val_env = TradingEnvironment(RL_INPUT_PATH / "rl_input_val.pkl",
                            max_steps_per_episode=MAX_STEPS)

# Create agent
print("\nCreating agent...")
agent = DoubleDQN(state_dim=28, action_dim=7, learning_rate=0.0005, 
                 gamma=0.95, epsilon_start=1.0, epsilon_end=0.01, 
                 epsilon_decay=0.995, buffer_size=50000)

# Training parameters
num_episodes = 200  # Reduced since each episode is now faster
target_update_freq = 100
batch_size = 64
validate_freq = 5  # Validate more frequently
early_stop_patience = 20

best_val_sharpe = -np.inf
no_improvement_count = 0
train_rewards = []
val_sharpes = []
val_pnls = []

print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)
print(f"Episodes: {num_episodes}")
print(f"Max steps per episode: {MAX_STEPS:,}")
print(f"Target update frequency: every {target_update_freq} steps")
print(f"Validation frequency: every {validate_freq} episodes")
print(f"Early stopping patience: {early_stop_patience} validations")
print("="*60 + "\n")

start_time = datetime.now()

for episode in range(num_episodes):
    episode_start = datetime.now()
    state = train_env.reset(random_start=True)
    episode_reward = 0
    steps = 0
    
    # Run episode with progress tracking
    while state is not None:
        action = agent.select_action(state, training=True)
        next_state, reward, done, _ = train_env.step(action)
        
        agent.remember(state, action, reward, next_state, done)
        
        # Train
        if len(agent.replay_buffer) >= batch_size:
            agent.train_step(batch_size=batch_size)
        
        state = next_state
        episode_reward += reward
        steps += 1
        
        if done:
            break
        
        # Update target network periodically
        if steps % target_update_freq == 0:
            agent.update_target_network()
    
    # Decay exploration
    agent.decay_epsilon()
    train_rewards.append(episode_reward)
    train_metrics = train_env.get_metrics()
    
    episode_time = (datetime.now() - episode_start).total_seconds()
    
    # Validate periodically
    if episode % validate_freq == 0:
        val_metrics = validate(agent, val_env)
        val_sharpes.append(val_metrics['sharpe'])
        val_pnls.append(val_metrics['total_pnl'])
        
        elapsed = (datetime.now() - start_time).total_seconds()
        
        print(f"\nEpisode {episode}/{num_episodes} (took {episode_time:.1f}s, total: {elapsed/60:.1f}min)")
        print(f"  Train - PnL: ${train_metrics['total_pnl']:.2f}, Sharpe: {train_metrics['sharpe']:.2f}, "
              f"Trades: {train_metrics['num_trades']}, Steps: {steps}")
        print(f"  Val   - PnL: ${val_metrics['total_pnl']:.2f}, Sharpe: {val_metrics['sharpe']:.2f}, "
              f"Trades: {val_metrics['num_trades']}")
        print(f"  Epsilon: {agent.epsilon:.3f}, Avg Loss: {np.mean(agent.losses[-100:]) if len(agent.losses) > 0 else 0:.4f}")
        
        # Save best model
        if val_metrics['sharpe'] > best_val_sharpe:
            best_val_sharpe = val_metrics['sharpe']
            no_improvement_count = 0
            agent.save(OUTPUT_PATH / "best_agent.pt")
            print(f"  ✓ New best model! (Sharpe: {best_val_sharpe:.2f})")
        else:
            no_improvement_count += 1
            if no_improvement_count >= early_stop_patience:
                print(f"\n⚠ Early stopping: No improvement for {early_stop_patience} validations")
                break

total_time = (datetime.now() - start_time).total_seconds()
print(f"\n⏱ Total training time: {total_time/60:.1f} minutes")

# ============================================================
# PLOT RESULTS
# ============================================================

print("\n" + "="*60)
print("PLOTTING TRAINING PROGRESS")
print("="*60)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Training rewards
axes[0, 0].plot(train_rewards)
axes[0, 0].set_title('Training Rewards per Episode')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Total Reward')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3)

# Validation Sharpe
val_episodes = list(range(0, len(train_rewards), validate_freq))[:len(val_sharpes)]
axes[0, 1].plot(val_episodes, val_sharpes, marker='o')
axes[0, 1].set_title('Validation Sharpe Ratio')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Sharpe Ratio')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.3)

# Validation PnL
axes[1, 0].plot(val_episodes, val_pnls, marker='o')
axes[1, 0].set_title('Validation PnL')
axes[1, 0].set_xlabel('Episode')
axes[1, 0].set_ylabel('PnL ($)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3)

# Training loss
if len(agent.losses) > 0:
    window = 50
    if len(agent.losses) > window:
        smoothed = np.convolve(agent.losses, np.ones(window)/window, mode='valid')
        axes[1, 1].plot(smoothed)
    else:
        axes[1, 1].plot(agent.losses)
    axes[1, 1].set_title('Training Loss (smoothed)')
    axes[1, 1].set_xlabel('Training Step')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_PATH / 'training_progress.png', dpi=150)
print(f"\n✓ Saved training plots to {OUTPUT_PATH / 'training_progress.png'}")

# ============================================================
# SUMMARY
# ============================================================

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)
print(f"\nBest validation Sharpe: {best_val_sharpe:.2f}")
print(f"Final training reward: ${train_rewards[-1]:.2f}")
print(f"Total episodes: {len(train_rewards)}")
print(f"Training time: {total_time/60:.1f} minutes")
print(f"Model saved to: {OUTPUT_PATH / 'best_agent.pt'}")
print("\n🎉 Ready for Step 4: Testing on Days 14-15!")